## Generates edges of graphs for users with common tweets that allows using "contagion" techniques to transmit the opinion from one to another

In [ ]:
modules = ["pymongo", "pprintpp", "rich","tqdm"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
  print("Instalando módulos")
  for m in modules:
      # para el import quitamos [...] y ==...
      p = m.find("[")
      mi = m if p==-1 else m[:p]
      p = mi.find("==")
      mi = mi if p==-1 else mi[:p]

      torch_loader = importlib.util.find_spec(mi)
      if torch_loader is not None:
          print(m," encontrado")
      else:
          print(m," No encontrado, instalando...",end="")  
          try:        
            r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
            print("¡hecho!")
          except:
            print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")
              
  print("¡Terminado!")

instala(modules) 

In [ ]:
import pandas as pd
user_file = "usa162024.csv"

path ="../data/"
import pandas as pd
df_user = pd.read_csv(user_file)

df_user[["User", "Tweet", "Date", "year", "party"]].to_csv("usa162024.csv",index=False)
df_user

In [ ]:
# change if needed, now it is done for a local server
url_servidor = 'mongodb://127.0.0.1:27017/'

# if it is in Atlas, use this line instead
#url_servidor= "mongodb+srv://example.mongodb.net/?retryWrites=true&w=majority"


from pymongo import MongoClient

# this is the connection to the server
client = MongoClient(url_servidor)

# code to check if the connection is ok
try:
    s = client.server_info() # if there is an error, it will raise an exception
    db = client.grafo
    print("Conectado a MongoDB, versión",s["version"])
except:
    print ("Error de conexión ¿está arrancado el servidor?")

In [ ]:
!mongoimport -d grafo -c user --headerline --type csv --drop --file "usa162024.csv"

In [ ]:
from rich import print_json
from json import dumps

# shows the content of a cursor in JSON format
def muestra(cursor):
    # Convert to JSON and print it
    i = 0
    for doc in cursor:
        i+=1
        print_json(dumps(doc, indent=4, default=str))
    return i

muestra(db.user.find().limit(10))

In [ ]:
db.user.drop_indexes()
db.aristas.drop_indexes()
db.aristas.drop()

resp = db.user.create_index([ ("Tweet",1)])
resp = db.aristas.create_index([ ("User1",1),("User2",1),("year",1) ],unique=True)
print ("index response:", resp)

In [ ]:
from tqdm import tqdm
tweets = db.user.distinct("Tweet")
aristas = []
for t in tqdm(tweets):
    df_t = pd.DataFrame(db.user.find({"Tweet":t},{"_id":0,"User":1,"year":1,"party":1}))
    for _,row1 in df_t .iterrows():
        for _,row2 in df_t .iterrows():
            if row1.User>row2.User and row1.year == row2.year and row1.party == row2.party :                                
                db.aristas.update_one({"User1":row1.User,"User2":row2.User, "year":row1.year},
                                      {"$setOnInsert":{"Tweet":t},"$inc":{"n":1}},upsert=True)


In [ ]:
!mongoexport -d grafo  -c aristas --type csv --fields "User1","User2","year","Tweet","n" --out "../data/edges_common.csv"

In [ ]:
data = "../data/"
df_edges = pd.read_csv(data + "edges_common.csv")
df_edges

#### Done without using MONGODB:

In [ ]:
from tqdm import tqdm
users = df_user.User.unique()
aristas = []
for y in [16,20,24]:
    for p in ["dem","rep"]:
        df_user_yp = df_user[(df_user.year==y) & (df_user.party==p)][["User", "Tweet", "Date", "party"]]
        print(y,p)
        for u1 in tqdm(users):
            df_u1 = df_user_yp[df_user_yp.User == u1]
            if len(df_u1)>0:
                for u2 in (users):
                    if u2>u1:
                        df_u2 = df_user_yp[df_user_yp.User == u2]
                        if len(df_u2)>0:
                            mezcla = df_u1.merge(df_u2,left_on="Tweet",right_on="Tweet")
                            if len(mezcla)>0:
                                aristas.append((u1,u2,mezcla.iloc[0]["Tweet"]))
                    
            